# 04 - Redes Neurais Recorrentes - LSTM - Estabilização e GPU

Vamos estabibilizar as previsões com uso de diversas previsões com seeds diferentes. Após tiramos a média e usamos como resultado. Também adicionamos comandos para execução do modelo em GPU.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('data-processed/ts_hr_feb_2020.csv', parse_dates=['date'])

data['hour'] = data['date'].dt.hour

last_sunday = pd.to_datetime("2020-02-23")
train = data[data['date'] < last_sunday]
val = data[data['date'] >= last_sunday]

In [3]:
def prep_seqs(df, l=24, h=24):
    X = []
    Y = []
    for i in range(l,df.shape[0]-h):
        f = df.iloc[i-l:i][['sales_value', 'hour']] # hoje
        y = df.iloc[i:i+h]['sales_value'] # 1 dia depois

        X.append(f.values)
        Y.append(y.values)

    X = np.array(X)
    Y = np.array(Y)

    #X = np.expand_dims(X, -1)
    X = np.swapaxes(X, 0,1)
    
    #Y = np.expand_dims(Y, -1)
    #Y = np.swapaxes(Y, 0,2)
    return torch.from_numpy(X).float(), torch.from_numpy(Y).float()

In [4]:
X, Y = prep_seqs(train)
Xt, Yt = prep_seqs(val)

mean_train = X.mean(1, keepdims=True)
mean_train[:, :, 1] = 0
std_train = X.std(1,  unbiased=False, keepdims=True)
std_train[:, :, 1] = 1

meanY_train = Y.mean(0, keepdims=True)
stdY_train = Y.std(0, unbiased=False, keepdims=True)

X -= mean_train
X /= std_train

Xt -= mean_train
Xt /= std_train


Y -= meanY_train
Y /= stdY_train


X = X.cuda()
Y = Y.cuda()

# cuda = torch.device('cuda') 
#Y.to(cuda)

In [5]:
class LSTMRNN(nn.Module):
    
    def __init__(self, hidden_size):
        super(LSTMRNN, self).__init__()
        self.hidden_size = hidden_size
        self.hour_emb = nn.Embedding(24, 2)
        self.lstm = nn.LSTM(3, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, 24)
        
    def forward(self, input, hidden=None):
        hour_emb = self.hour_emb(input[:,:,-1].long().cuda())
        nums = input[:,:,:-1].cuda()
        inputs = torch.cat([nums, hour_emb], dim=2)
        
        output, _ = self.lstm(inputs)
        output = output[-1, :, :]
        
        output = self.out(output)
        return output.squeeze()

In [6]:
total_p = np.zeros((Yt.shape[0], Yt.shape[1], 10))

for seed in range(10):
    torch.manual_seed(seed)
    np.random.seed(seed)

    lstm = LSTMRNN(100).cuda()
    criterion = nn.L1Loss()
    lstm_optimizer = optim.Adam(lstm.parameters(), lr=1e-2)

    for i in range(100):
        lstm.zero_grad()

        o = lstm(X)
        loss = criterion(o, Y)
        loss.backward()
        lstm_optimizer.step()
        #print(loss.item())

        p = lstm(Xt)
        p = p.detach().cpu()

        p *= stdY_train
        p += meanY_train
        p = p.numpy().squeeze()


    print("Seed = {} - Erro: {}".format(seed, mean_absolute_error(Yt.numpy(), p)))
    total_p[:, :, seed] = p
    
print("\nEnsemble Avg Score = {}\n".format(mean_absolute_error(Yt.numpy(), total_p.mean(axis=-1))))

#mesma seed na GPU dá resultados diferentes

Seed = 0 - Erro: 345.7042541503906
Seed = 1 - Erro: 342.533203125
Seed = 2 - Erro: 354.215087890625
Seed = 3 - Erro: 347.9766540527344
Seed = 4 - Erro: 352.1267395019531
Seed = 5 - Erro: 346.0849914550781
Seed = 6 - Erro: 355.3306579589844
Seed = 7 - Erro: 340.03515625
Seed = 8 - Erro: 354.319091796875
Seed = 9 - Erro: 348.231201171875

Ensemble Avg Score = 342.096002009158



# Fim